In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn import svm
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.decomposition import FastICA
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import precision_recall_fscore_support

In [2]:
#df = pd.read_csv("heart.csv")
#df = pd.read_csv("cardio.csv")
#df = pd.read_csv("framingham.csv")
df = pd.read_csv("filtered_framingham.csv")
#df = pd.read_csv("balanced_framingham.csv")

In [3]:
df

,Unnamed: 0,male,age,education,currentSmoker,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,BMI,heartRate,TenYearCHD
0,0,1,39,4.0,0,0.0,0,0,0,195.0,26.97,80.0,0
1,1,0,46,2.0,0,0.0,0,0,0,250.0,28.73,95.0,0
2,2,1,48,1.0,1,0.0,0,0,0,245.0,25.34,75.0,0
3,3,0,61,3.0,1,0.0,0,1,0,225.0,28.58,65.0,1
4,4,0,46,3.0,1,0.0,0,0,0,285.0,23.10,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,4235,0,48,2.0,1,0.0,0,0,0,248.0,22.00,84.0,0
4236,4236,0,44,1.0,1,0.0,0,0,0,210.0,19.16,86.0,0
4237,4237,0,52,2.0,0,0.0,0,0,0,269.0,21.47,80.0,0
4238,4238,1,40,3.0,0,0.0,0,1,0,185.0,25.60,67.0,0


In [4]:
#df =df.drop(['id','age'],axis = 1)
df = df.drop(['Unnamed: 0'], axis = 1)

In [5]:
df

,male,age,education,currentSmoker,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,BMI,heartRate,TenYearCHD
0,1,39,4.0,0,0.0,0,0,0,195.0,26.97,80.0,0
1,0,46,2.0,0,0.0,0,0,0,250.0,28.73,95.0,0
2,1,48,1.0,1,0.0,0,0,0,245.0,25.34,75.0,0
3,0,61,3.0,1,0.0,0,1,0,225.0,28.58,65.0,1
4,0,46,3.0,1,0.0,0,0,0,285.0,23.10,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,2.0,1,0.0,0,0,0,248.0,22.00,84.0,0
4236,0,44,1.0,1,0.0,0,0,0,210.0,19.16,86.0,0
4237,0,52,2.0,0,0.0,0,0,0,269.0,21.47,80.0,0
4238,1,40,3.0,0,0.0,0,1,0,185.0,25.60,67.0,0


In [6]:
# finding the null values in dataset for framingham dataset
df.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
BMI                0
heartRate          0
TenYearCHD         0
dtype: int64

In [5]:
# filling the datset with missing values median
df['education'] = df['education'].fillna(df['education'].median())
df['cigsPerDay'] = df['cigsPerDay'].fillna(df['cigsPerDay'].median())
df['BPMeds'] = df['BPMeds'].fillna(df['BPMeds'].median())
df['totChol'] = df['totChol'].fillna(df['totChol'].median())
df['BMI'] = df['BMI'].fillna(df['BMI'].median())
df['heartRate'] = df['heartRate'].fillna(df['heartRate'].median())
df['glucose'] = df['glucose'].fillna(df['glucose'].median())

In [7]:
df.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
BMI                0
heartRate          0
TenYearCHD         0
dtype: int64

In [7]:
#converting all the datatypes into float
df = df.astype(float)

In [8]:
df.round(3)

,male,age,education,currentSmoker,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,BMI,heartRate,TenYearCHD
0,1.0,39.0,4.0,0.0,0.0,0.0,0.0,0.0,195.0,26.97,80.0,0.0
1,0.0,46.0,2.0,0.0,0.0,0.0,0.0,0.0,250.0,28.73,95.0,0.0
2,1.0,48.0,1.0,1.0,0.0,0.0,0.0,0.0,245.0,25.34,75.0,0.0
3,0.0,61.0,3.0,1.0,0.0,0.0,1.0,0.0,225.0,28.58,65.0,1.0
4,0.0,46.0,3.0,1.0,0.0,0.0,0.0,0.0,285.0,23.10,85.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0.0,48.0,2.0,1.0,0.0,0.0,0.0,0.0,248.0,22.00,84.0,0.0
4236,0.0,44.0,1.0,1.0,0.0,0.0,0.0,0.0,210.0,19.16,86.0,0.0
4237,0.0,52.0,2.0,0.0,0.0,0.0,0.0,0.0,269.0,21.47,80.0,0.0
4238,1.0,40.0,3.0,0.0,0.0,0.0,1.0,0.0,185.0,25.60,67.0,0.0


In [9]:
X_FEATURES = ['male','age','education','currentSmoker','cigePerDay','BPMeds','prevalentStroke','prevalentHyp','diabetes','totChol','sysBP','diaBP'
              ,'BMI','heartRate','glucose']
x_features = df.loc[:,X_FEATURES].values.astype(int)
y_features = df.loc[:,['TenYearCHD']].values.astype(int)

C:\Users\Praty\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [9]:
X_FEATURES = ['male','age','education','currentSmoker','BPMeds','prevalentStroke','prevalentHyp','diabetes','totChol'
              ,'BMI','heartRate']
x_features = df.loc[:,X_FEATURES].values.astype(int)
y_features = df.loc[:,['TenYearCHD']].values.astype(int)

In [ ]:
X_FEATURES = ['gender','height','weight','ap_hi','ap_lo','cholesterol','gluc','smoke','alco','active']
x_features = df.loc[:,X_FEATURES].values.astype(int)
y_features = df.loc[:,['cardio']].values.astype(int)

In [ ]:
X_FEATURES = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca']
x_features = df.loc[:,X_FEATURES].values.astype(int)
y_features = df.loc[:,['target']].values.astype(int)

# RANDOM FOREST CLASSIFICATION

In [11]:
def forest_train(x_features,y_features):
    X_Train, X_Test, Y_Train, Y_Test = model_selection.train_test_split(x_features, y_features, test_size=0.20, random_state = 101)
    Random_Forest = RandomForestClassifier(n_estimators = 100, max_depth = 6, min_samples_split = 2, random_state = 0).fit(X_Train,Y_Train)
    Random_Forest.fit(X_Train, Y_Train)
    predict_Forest = Random_Forest.predict(X_Test)
    print(" ",metrics.accuracy_score(Y_Test,predict_Forest))
    print(confusion_matrix(Y_Test, predict_Forest))
    print(classification_report(Y_Test, predict_Forest))
    print("recall precision f1-score")
    print(precision_recall_fscore_support(Y_Test, predict_Forest, average='weighted'))

In [12]:
forest_train(x_features,y_features)

C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


  0.8691037735849056
[[737   0]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

recall precision f1-score
(0.755341369259523, 0.8691037735849056, 0.8082390929111362, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
#ICA
ica = FastICA(n_components = 2)
X_ica = ica.fit_transform(x_features)
forest_train(X_ica,y_features)

C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


  0.8667452830188679
[[735   2]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

recall precision f1-score
(0.7550724274053258, 0.8667452830188679, 0.8070641485595778, None)


In [14]:
#PCA
pca = PCA(n_components = 3, svd_solver = 'full')
X_pca = pca.fit_transform(x_features)
print(pca.explained_variance_)
forest_train(X_pca, y_features)

[1971.61509236  143.5636573    68.46249996]


C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


  0.8691037735849056
[[737   0]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

recall precision f1-score
(0.755341369259523, 0.8691037735849056, 0.8082390929111362, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
#LDA
lda = LinearDiscriminantAnalysis(n_components = 1)
x_lda = lda.fit(x_features, y_features).transform(x_features)
print('Original number of features:', x_features.shape[1])
print('Reduced number of features:', x_lda.shape[1])

Original number of features: 11
Reduced number of features: 1


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
forest_train(x_lda, y_features)

C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


  0.8667452830188679
[[735   2]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

recall precision f1-score
(0.7550724274053258, 0.8667452830188679, 0.8070641485595778, None)


In [ ]:
ICA_df = pd.DataFrame(data = X_ica, columns = ['IC1', 'IC2'])
ICA_df = pd.concat([ICA_df, df['TenYearCHD']], axis = 1)
ICA_df['TenYearCHD'] = LabelEncoder().fit_transform(ICA_df['TenYearCHD'])
ICA_df.head()

# SUPPORT VECTOR MACHINE

In [33]:
def SVM_train(x_features,y_features):
    X_Train, X_Test, Y_Train, Y_Test = model_selection.train_test_split(x_features, y_features, test_size=0.20, random_state = 101)
    SVM_MODEL = svm.SVC(kernel = 'linear', C = 1, degree = 3, gamma = 'auto').fit(X_Train,Y_Train)
    SVM_MODEL.fit(X_Train, Y_Train)
    predict_SVM = SVM_MODEL.predict(X_Test)
    print(" ",metrics.accuracy_score(Y_Test,predict_SVM))
    print(confusion_matrix(Y_Test, predict_SVM))
    print(classification_report(Y_Test, predict_SVM))
    print(precision_recall_fscore_support(Y_Test, predict_SVM, average='weighted'))

In [34]:
SVM_train(x_features,y_features)

C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


  0.8691037735849056
[[737   0]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

(0.755341369259523, 0.8691037735849056, 0.8082390929111362, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [19]:
ica = FastICA(n_components = 3)
X1_ica = ica.fit_transform(x_features)
SVM_train(X1_ica,y_features)

  0.8691037735849056
[[737   0]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

(0.755341369259523, 0.8691037735849056, 0.8082390929111362, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
pca = PCA(n_components = 3, svd_solver = 'full')
X1_pca = pca.fit_transform(x_features)
print(pca.explained_variance_)
SVM_train(X1_pca, y_features)

[1971.61509236  143.5636573    68.46249996]


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [35]:
#LDA
lda = LinearDiscriminantAnalysis(n_components = 1)
x1_lda = lda.fit(x_features, y_features).transform(x_features)
print('Original number of features:', x_features.shape[1])
print('Reduced number of features:', x1_lda.shape[1])
SVM_train(x1_lda, y_features)

Original number of features: 11
Reduced number of features: 1
  0.8691037735849056
[[737   0]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

(0.755341369259523, 0.8691037735849056, 0.8082390929111362, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  

# DECISION TREE CLASSIFER

In [10]:
def DT_train(x_features,y_features):
    X_Train, X_Test, Y_Train, Y_Test = model_selection.train_test_split(x_features, y_features, test_size=0.20, random_state = 101)
    Decision_Tree_Classifier = DecisionTreeClassifier( max_depth=6, min_samples_split=2,random_state=0).fit(X_Train,Y_Train)
    Decision_Tree_Classifier.fit(X_Train, Y_Train)
    predict_DT = Decision_Tree_Classifier.predict(X_Test)
    print(" ",metrics.accuracy_score(Y_Test,predict_DT))
    print(confusion_matrix(Y_Test, predict_DT))
    print(classification_report(Y_Test, predict_DT))
    print(precision_recall_fscore_support(Y_Test, predict_DT, average='weighted'))

In [11]:
DT_train(x_features,y_features)

  0.8537735849056604
[[720  17]
 [107   4]]
              precision    recall  f1-score   support

           0       0.87      0.98      0.92       737
           1       0.19      0.04      0.06       111

    accuracy                           0.85       848
   macro avg       0.53      0.51      0.49       848
weighted avg       0.78      0.85      0.81       848

(0.7815888624163589, 0.8537735849056604, 0.808130952433177, None)


In [12]:
ica = FastICA(n_components = 3)
X2_ica = ica.fit_transform(x_features)
DT_train(X2_ica,y_features)

  0.8632075471698113
[[728   9]
 [107   4]]
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       737
           1       0.31      0.04      0.06       111

    accuracy                           0.86       848
   macro avg       0.59      0.51      0.50       848
weighted avg       0.80      0.86      0.81       848

(0.798009351398799, 0.8632075471698113, 0.813416351891516, None)


In [13]:
pca = PCA(n_components = 3, svd_solver = 'full')
X2_pca = pca.fit_transform(x_features)
print(pca.explained_variance_)
DT_train(X2_pca, y_features)

[1971.61509236  143.5636573    68.46249996]
  0.8632075471698113
[[730   7]
 [109   2]]
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       737
           1       0.22      0.02      0.03       111

    accuracy                           0.86       848
   macro avg       0.55      0.50      0.48       848
weighted avg       0.79      0.86      0.81       848

(0.7852808449711771, 0.8632075471698113, 0.8094974140408008, None)


In [14]:
#LDA
lda = LinearDiscriminantAnalysis(n_components = 1)
x2_lda = lda.fit(x_features, y_features).transform(x_features)
print('Original number of features:', x_features.shape[1])
print('Reduced number of features:', x2_lda.shape[1])
DT_train(x2_lda, y_features)

Original number of features: 11
Reduced number of features: 1
  0.8596698113207547
[[729   8]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      0.99      0.92       737
           1       0.00      0.00      0.00       111

    accuracy                           0.86       848
   macro avg       0.43      0.49      0.46       848
weighted avg       0.75      0.86      0.80       848

(0.7542579177897574, 0.8596698113207547, 0.8035214342972685, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# EXTRA TREE CLASSIFIER

In [15]:
def ET_train(x_features,y_features):
    X_Train, X_Test, Y_Train, Y_Test = model_selection.train_test_split(x_features, y_features, test_size=0.20, random_state = 101)
    Extra_Tree_Classifier = ExtraTreesClassifier(n_estimators = 100, max_depth=6, min_samples_split=2,random_state=0).fit(X_Train,Y_Train)
    Extra_Tree_Classifier.fit(X_Train, Y_Train)
    predict_ET = Extra_Tree_Classifier.predict(X_Test)
    print(" ",metrics.accuracy_score(Y_Test,predict_ET))
    print(confusion_matrix(Y_Test, predict_ET))
    print(classification_report(Y_Test, predict_ET))
    print(precision_recall_fscore_support(Y_Test, predict_ET, average='weighted'))

In [16]:
ET_train(x_features,y_features)

C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


  0.8691037735849056
[[736   1]
 [110   1]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.50      0.01      0.02       111

    accuracy                           0.87       848
   macro avg       0.68      0.50      0.47       848
weighted avg       0.82      0.87      0.81       848

(0.8215478500379143, 0.8691037735849056, 0.8104789423903815, None)


In [17]:
ica = FastICA(n_components = 3)
X3_ica = ica.fit_transform(x_features)
ET_train(X3_ica,y_features)

C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


  0.8691037735849056
[[737   0]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

(0.755341369259523, 0.8691037735849056, 0.8082390929111362, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
pca = PCA(n_components = 3, svd_solver = 'full')
X3_pca = pca.fit_transform(x_features)
print(pca.explained_variance_)
ET_train(X3_pca, y_features)

[1971.61509236  143.5636573    68.46249996]


C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


  0.8691037735849056
[[737   0]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

(0.755341369259523, 0.8691037735849056, 0.8082390929111362, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [19]:
#LDA
lda = LinearDiscriminantAnalysis(n_components = 1)
x3_lda = lda.fit(x_features, y_features).transform(x_features)
print('Original number of features:', x_features.shape[1])
print('Reduced number of features:', x3_lda.shape[1])
ET_train(x3_lda, y_features)

C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Original number of features: 11
Reduced number of features: 1
  0.8667452830188679
[[735   2]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

(0.7550724274053258, 0.8667452830188679, 0.8070641485595778, None)


# XGBOOST

In [20]:
import xgboost as xgb

In [21]:
def XG_train(x_features,y_features):
    X_Train, X_Test, Y_Train, Y_Test = model_selection.train_test_split(x_features, y_features, test_size=0.20, random_state = 101)
    XGBOOST_Classifier = xgb.XGBClassifier(random_state=0, learning_rate = 0.01).fit(X_Train,Y_Train)
    XGBOOST_Classifier.fit(X_Train, Y_Train)
    predict_XG = XGBOOST_Classifier.predict(X_Test)
    print(" ",metrics.accuracy_score(Y_Test,predict_XG))
    print(confusion_matrix(Y_Test, predict_XG))
    print(classification_report(Y_Test, predict_XG))
    print(precision_recall_fscore_support(Y_Test, predict_XG, average='weighted'))

In [22]:
XG_train(x_features, y_features)

C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


  0.8584905660377359
[[726  11]
 [109   2]]
              precision    recall  f1-score   support

           0       0.87      0.99      0.92       737
           1       0.15      0.02      0.03       111

    accuracy                           0.86       848
   macro avg       0.51      0.50      0.48       848
weighted avg       0.78      0.86      0.81       848

(0.775789784726628, 0.8584905660377359, 0.8069824329912234, None)


In [25]:
##### ica = FastICA(n_components = 3)
X4_ica = ica.fit_transform(x_features)
XG_train(X4_ica,y_features)

C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


  0.8655660377358491
[[732   5]
 [109   2]]
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       737
           1       0.29      0.02      0.03       111

    accuracy                           0.87       848
   macro avg       0.58      0.51      0.48       848
weighted avg       0.79      0.87      0.81       848

(0.793860232491803, 0.8655660377358491, 0.8107539691707575, None)


In [26]:
##### pca = PCA(n_components = 3, svd_solver = 'full')
X4_pca = pca.fit_transform(x_features)
print(pca.explained_variance_)
XG_train(X4_pca, y_features)

[1971.61509236  143.5636573    68.46249996]


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


  0.8667452830188679
[[734   3]
 [110   1]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.25      0.01      0.02       111

    accuracy                           0.87       848
   macro avg       0.56      0.50      0.47       848
weighted avg       0.79      0.87      0.81       848

(0.7885560113565234, 0.8667452830188679, 0.8092621231720611, None)


In [24]:
#LDA
lda = LinearDiscriminantAnalysis(n_components = 1)
x4_lda = lda.fit(x_features, y_features).transform(x_features)
print('Original number of features:', x_features.shape[1])
print('Reduced number of features:', x4_lda.shape[1])
XG_train(x4_lda, y_features)

C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Original number of features: 11
Reduced number of features: 1
  0.8667452830188679
[[735   2]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

(0.7550724274053258, 0.8667452830188679, 0.8070641485595778, None)


# K NEAREST NEIGHBOURS

In [27]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [28]:
def KNN_train(x_features,y_features):
    X_Train, X_Test, Y_Train, Y_Test = model_selection.train_test_split(x_features, y_features, test_size=0.20, random_state = 101)
    KNN_Classifier = BaggingClassifier(KNeighborsClassifier(),max_samples=0.4, max_features=0.6).fit(X_Train, Y_Train)
    KNN_Classifier.fit(X_Train, Y_Train)
    predict_KNN = KNN_Classifier.predict(X_Test)
    print(" ",metrics.accuracy_score(Y_Test,predict_KNN))
    print(confusion_matrix(Y_Test, predict_KNN))
    print(classification_report(Y_Test, predict_KNN))
    print(precision_recall_fscore_support(Y_Test, predict_KNN, average='weighted'))

In [29]:
KNN_train(x_features, y_features)

  0.8691037735849056
[[737   0]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

(0.755341369259523, 0.8691037735849056, 0.8082390929111362, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [30]:
ica = FastICA(n_components = 3)
X5_ica = ica.fit_transform(x_features)
KNN_train(X5_ica,y_features)

  0.8691037735849056
[[737   0]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

(0.755341369259523, 0.8691037735849056, 0.8082390929111362, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [31]:
pca = PCA(n_components = 3, svd_solver = 'full')
X5_pca = pca.fit_transform(x_features)
print(pca.explained_variance_)
KNN_train(X5_pca, y_features)

[1971.61509236  143.5636573    68.46249996]
  0.8691037735849056
[[737   0]
 [111   0]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       737
           1       0.00      0.00      0.00       111

    accuracy                           0.87       848
   macro avg       0.43      0.50      0.46       848
weighted avg       0.76      0.87      0.81       848

(0.755341369259523, 0.8691037735849056, 0.8082390929111362, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [32]:
#LDA
lda = LinearDiscriminantAnalysis(n_components = 1)
x5_lda = lda.fit(x_features, y_features).transform(x_features)
print('Original number of features:', x_features.shape[1])
print('Reduced number of features:', x5_lda.shape[1])
KNN_train(x5_lda, y_features)

Original number of features: 11
Reduced number of features: 1
  0.8596698113207547
[[728   9]
 [110   1]]
              precision    recall  f1-score   support

           0       0.87      0.99      0.92       737
           1       0.10      0.01      0.02       111

    accuracy                           0.86       848
   macro avg       0.48      0.50      0.47       848
weighted avg       0.77      0.86      0.81       848

(0.7681105619849597, 0.8596698113207547, 0.8056017291266004, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# ann model

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
sc = StandardScaler()
X_train, X_test, Y_train, Y_test = train_test_split(x_features, y_features, test_size = 0.2, random_state = 0)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
classifier = Sequential()

In [ ]:
classifier.add(Dense(output_dim = 18, init = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dense(output_dim = 10, init = 'uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.fit(X_train, Y_train, batch_size = 10 , nb_epoch = 1000)

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
print(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
print(cm)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_pred, Y_test)

In [ ]:
accuracy